In [1]:
# KNN - content based approach, 
# where for each movie we find K similar movies (nearest neighbours) to it, where the similarity is Cosine Similarity

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:/Users/Priyanka/Desktop/PGP-DSE/Capstoneprojectworks/main_dataframe.csv')
df.head(2)

,adult,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,genre_name,collection_name,production_companies_name,keywords,cast_name,crew_name,vote_average
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,"['Animation', 'Comedy', 'Family']",Toy Story Collection,['Pixar Animation Studios'],"['jealousy', 'toy', 'boy', 'friendship', 'frie...","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...","['John Lasseter', 'Joss Whedon', 'Andrew Stant...",7.7
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,"['Adventure', 'Fantasy', 'Family']",NaN,"['TriStar Pictures', 'Teitler Film', 'Intersco...","['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D...","['Larry J. Franco', 'Jonathan Hensleigh', 'Jam...",6.9


In [3]:
df.columns

Index(['adult', 'id', 'original_language', 'original_title', 'overview',
       'popularity', 'release_date', 'revenue', 'runtime', 'status',
       'genre_name', 'collection_name', 'production_companies_name',
       'keywords', 'cast_name', 'crew_name', 'vote_average'],
      dtype='object')

In [4]:
# columns taken into account for => dependent variable ["original_title"]
df_data = df[['id','original_title','genre_name','cast_name','keywords','vote_average']]
df_data.head(1)

,id,original_title,genre_name,cast_name,keywords,vote_average
0,862,Toy Story,"['Animation', 'Comedy', 'Family']","['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...","['jealousy', 'toy', 'boy', 'friendship', 'frie...",7.7


In [5]:
df_data.shape

(45463, 6)

In [6]:
df_data.iloc[0]

id                                                              862
original_title                                            Toy Story
genre_name                        ['Animation', 'Comedy', 'Family']
cast_name         ['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...
keywords          ['jealousy', 'toy', 'boy', 'friendship', 'frie...
vote_average                                                    7.7
Name: 0, dtype: object

In [7]:
for i,j in zip(df_data['genre_name'],df_data.index):
    list2=[]
    list2=i
    # list2.sort()
    df_data.loc[j,'genre_name']=str(list2)
df_data['genre_name'] = df_data['genre_name'].str.strip('[]').str.replace(' ','').str.replace("'",'')
df_data['genre_name'] = df_data['genre_name'].str.split(',')

c:\Users\Priyanka\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/1643974628.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['genre_name'] = df_data['genre_name'].str.strip('[]').str.replace(' ','').str.replace("'",'')
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/1643974628.py:7: SettingWithCopyWarning: 
A value is trying to be set on a 

In [8]:
# Trying to find a list of unique genres

genreList = []
for index, row in df_data.iterrows():
    genres = row["genre_name"]
    
    for genre in genres:
        if genre not in genreList:
            genreList.append(genre)
genreList[:10]

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller']

In [9]:
# converting genre_name,keywords,cast_name into binary lists
def binary(genre_list):
    binaryList = []
    
    for genre in genreList:
        if genre in genre_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList

df_data['genres_bin'] = df_data['genre_name'].apply(lambda x: binary(x))


C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/1098395291.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['genres_bin'] = df_data['genre_name'].apply(lambda x: binary(x))


In [10]:
df_data['genres_bin'].head(3)

0    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: genres_bin, dtype: object

In [11]:
df_data.isna().sum()

id                    0
original_title        0
genre_name            0
cast_name             1
keywords          14353
vote_average          3
genres_bin            0
dtype: int64

In [12]:
df_data['keywords'] = df_data['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
df_data['keywords'] = df_data['keywords'].str.split(',')
for i,j in zip(df_data['keywords'],df_data.index):
    list2 = []
    list2 = i
    df_data.loc[j,'keywords'] = str(list2)
df_data['keywords'] = df_data['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
df_data['keywords'] = df_data['keywords'].str.split(',')
for i,j in zip(df_data['keywords'],df_data.index):
    list2 = []
    list2 = i
    # list2.sort()
    df_data.loc[j,'keywords'] = str(list2)
df_data['keywords'] = df_data['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'')
df_data['keywords'] = df_data['keywords'].str.split(',')

C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/500805389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['keywords'] = df_data['keywords'].str.strip('[]').str.replace(' ','').str.replace("'",'').str.replace('"','')
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/500805389.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['keywords'] = df_data['keywords'].str.split(',')
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/500805389.py:7: SettingWithCopyWarning: 
A value is tr

In [13]:
words_list = []
for index, row in df_data.iterrows():
    genres = row["keywords"]
    
    for genre in genres:
        if genre not in words_list:
            words_list.append(genre)


In [14]:
def binary(words):
    binaryList = []
    for genre in words_list:
        if genre in words:
            binaryList.append(1)
        else:
            binaryList.append(0)
    return binaryList
df_data['words_bin'] = df_data['keywords'].apply(lambda x: binary(x))

C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/2192393038.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['words_bin'] = df_data['keywords'].apply(lambda x: binary(x))


In [15]:
df_data['words_bin'].head(3)

0    [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: words_bin, dtype: object

In [16]:
df_data.dropna(inplace=True)

c:\Users\Priyanka\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [18]:

for i,j in zip(df_data['cast_name'],df_data.index):
    list2 = []
    list2 = i[:4]
    df_data.loc[j,'cast_name'] = str(list2)
df_data['cast_name'] = df_data['cast_name'].str.strip('[]').str.replace(' ','').str.replace("'",'')
df_data['cast_name'] = df_data['cast_name'].str.split(',')
for i,j in zip(df_data['cast_name'],df_data.index):
    list2 = []
    list2 = i
    #list2.sort()
    df_data.loc[j,'cast_name'] = str(list2)
df_data['cast_name']=df_data['cast_name'].str.strip('[]').str.replace(' ','').str.replace("'",'')

c:\Users\Priyanka\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/3780366754.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['cast_name'] = df_data['cast_name'].str.strip('[]').str.replace(' ','').str.replace("'",'')
C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/3780366754.py:6: SettingWithCopyWarning: 
A value is trying to be set on a co

In [19]:
castList = []
for index, row in df_data.iterrows():
    cast = row["cast_name"]
    
    for i in cast:
        if i not in castList:
            castList.append(i)
        

In [20]:
def binary(cast_list):
    binaryList = []
    
    for genre in castList:
        if genre in cast_list:
            binaryList.append(1)
        else:
            binaryList.append(0)
    
    return binaryList
df_data['cast_bin'] = df_data['cast_name'].apply(lambda x: binary(x))


C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/3379228768.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['cast_bin'] = df_data['cast_name'].apply(lambda x: binary(x))


In [21]:
df_data['cast_bin'].head()

0    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...
Name: cast_bin, dtype: object

In [22]:
new_id = list(range(0,df_data.shape[0]))
df_data['new_id']=new_id

C:\Users\Priyanka\AppData\Local\Temp/ipykernel_25584/3507510346.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['new_id']=new_id


In [23]:
df_data.columns

Index(['id', 'original_title', 'genre_name', 'cast_name', 'keywords',
       'vote_average', 'genres_bin', 'words_bin', 'cast_bin', 'new_id'],
      dtype='object')

In [24]:
# similarity function [ To find the distance] -> For movieid,genre,words,cast
from scipy import spatial

def Similarity(movieId1, movieId2):
    a = df_data.iloc[movieId1]
    b = df_data.iloc[movieId2]
    
    genresA = a['genres_bin']
    genresB = b['genres_bin']
    
    genreDistance = spatial.distance.cosine(genresA, genresB)
    
    wordsA = a['words_bin']
    wordsB = b['words_bin']
    wordsDistance = spatial.distance.cosine(wordsA, wordsB)

    castA = a['cast_bin']
    castB = b['cast_bin']
    castDistance = spatial.distance.cosine(castA, castB)

    return genreDistance + wordsDistance + castDistance

In [26]:
import operator

def scorePredictor(name):
    #name = input('Enter a movie title: ')
    new_movie = df_data[df_data['original_title'].str.contains(name)].iloc[0].to_frame().T
    print('Selected Movie: ',new_movie.original_title.values[0])
    def getNeighbors(baseMovie, K):
        distances = []
    
        for index, movie in df_data.iterrows():
            if movie['new_id'] != baseMovie['new_id'].values[0]:
                dist = Similarity(baseMovie['new_id'].values[0], movie['new_id'])
                distances.append((movie['new_id'], dist))
    
        distances.sort(key=operator.itemgetter(1))
        neighbors = []
    
        for x in range(K):
            neighbors.append(distances[x])
        return neighbors

    K = 12
    avgRating = 0
    neighbors = getNeighbors(new_movie, K)
    
    print('\nThe Recommended Movies: \n')
    for neighbor in neighbors:
        avgRating = avgRating+df_data.iloc[neighbor[0]][5] 
        print( df_data.iloc[neighbor[0]][1]+" | Genres: "+str(df_data.iloc[neighbor[0]][2]).strip('[]').replace(' ',''))

# Predicted rating vs actual rating 

    print('\n')
    avgRating = avgRating/K
    print('The predicted rating for %s is: %f' %(new_movie['original_title'].values[0],avgRating))
    print('The actual rating for %s is %f' %(new_movie['original_title'].values[0],new_movie['vote_average']))


# First Lets try giving the name via the function and then make it genric
scorePredictor('Stuart Little')

Selected Movie:  Stuart Little

The Recommended Movies: 

Shrek the Halls | Genres: 'Adventure','Animation','Comedy','Fantasy','Family'
借りぐらしのアリエッティ | Genres: 'Fantasy','Animation','Family'
Lorenzo | Genres: 'Animation','Comedy','Family'
Mr Hublot | Genres: 'Animation','Comedy','Family','Fantasy'
Ух ты, говорящая рыба! | Genres: 'Comedy','Animation','Fantasy','Family'
The Cat in the Hat | Genres: 'Comedy','Fantasy','Family'
Space Jam | Genres: 'Animation','Comedy','Drama','Family','Fantasy'
Shrek | Genres: 'Adventure','Animation','Comedy','Family','Fantasy'
Shrek 2 | Genres: 'Adventure','Animation','Comedy','Family','Fantasy'
Shrek the Third | Genres: 'Fantasy','Adventure','Animation','Comedy','Family'
Shrek Forever After | Genres: 'Comedy','Adventure','Fantasy','Animation','Family'
Stuart Little 3: Call of the Wild | Genres: 'Fantasy','Animation','Comedy','ScienceFiction','Family'


The predicted rating for Stuart Little is: 6.458333
The actual rating for Stuart Little is 5.800000


In [ ]:
# Hyperparameter Tuning :
# 1. GS / BO optimization
# 2. NN / Shrinkage factors
# 3. Ref - https://towardsdatascience.com/recommender-systems-and-hyper-parameter-tuning-25567b10e298
